In [33]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re
import os  # <-- thêm

def crawl_cafeland_region(url_template, max_pages=5):
    """
    Crawl dữ liệu bất động sản từ Cafeland theo khu vực và lưu CSV
    :param url_template: URL khu vực với {page}, ví dụ
                         "https://nhadat.cafeland.vn/nha-dat-ban-tai-son-la/?page={}"
    :param max_pages: số trang muốn crawl
    """
    # Lấy tên khu vực từ URL để đặt tên CSV
    match = re.search(r"nha-dat-ban-tai-(.*?)/", url_template)
    region_name = match.group(1).replace("-", "_") if match else "unknown"

    # Tạo thư mục nếu chưa tồn tại
    output_dir = "/content/drive/MyDrive/Mining/data"
    os.makedirs(output_dir, exist_ok=True)

    # Đường dẫn CSV
    csv_filename = os.path.join(output_dir, f"property_{region_name}.csv")

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/142.0.0.0 Safari/537.36"
    }

    all_data = []

    for page in range(1, max_pages + 1):
        print(f"Fetching page {page} ...")
        url = url_template.format(page)
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Không thể truy cập page {page}, status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        listings = soup.find_all("a", class_="realTitle")

        for a in listings:
            link = a['href']
            print(f"Processing {link}")

            # Truy cập chi tiết
            listing_resp = requests.get(link, headers=headers)
            if listing_resp.status_code != 200:
                print(f"Không truy cập được link {link}")
                continue

            listing_soup = BeautifulSoup(listing_resp.text, "html.parser")

            info = {
                "Diện tích": "",
                "Phòng ngủ": "",
                "Số tầng": "",
                "Số toilet": "",
                "Số phòng khách": "",
                "Loại địa ốc": "",
                "Địa chỉ": "",
                "Pháp lý": "",
                "Hướng nhà": "",
                "Đường trước nhà": "",
                "Giá nhà": ""
            }

            # span.title-item + value-item
            for span in listing_soup.find_all("span", class_="title-item"):
                key = span.get_text(strip=True)
                if key in info:
                    value_span = span.find_next_sibling("span", class_="value-item")
                    if value_span:
                        info[key] = value_span.get_text(strip=True)

            # div.infor-note + infor-data
            for note in listing_soup.find_all("div", class_="infor-note"):
                label = note.get_text(strip=True)
                if label in info:
                    data_div = note.find_next_sibling("div", class_="infor-data")
                    if data_div:
                        info[label] = data_div.get_text(strip=True)

            # Giá nhà
            for div in listing_soup.find_all("div", class_="infor-data"):
                text = div.get_text(strip=True)
                if "triệu" in text.lower() or "tỷ" in text.lower():
                    info["Giá nhà"] = text
                    break

            all_data.append([
                info["Diện tích"],
                info["Phòng ngủ"],
                info["Số tầng"],
                info["Số toilet"],
                info["Số phòng khách"],
                info["Loại địa ốc"],
                info["Địa chỉ"],
                info["Pháp lý"],
                info["Hướng nhà"],
                info["Đường trước nhà"],
                info["Giá nhà"]
            ])

            time.sleep(0.5)  # tránh chặn server

    # Lưu CSV
    with open(csv_filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            "Diện tích","Phòng ngủ","Số tầng","Số toilet","Số phòng khách",
            "Loại địa ốc","Địa chỉ","Pháp lý","Hướng nhà","Đường trước nhà","Giá nhà"
        ])
        writer.writerows(all_data)

    print(f"Đã lưu {len(all_data)} bất động sản vào {csv_filename}")

In [35]:
import requests
from bs4 import BeautifulSoup

def get_total_pages_dynamic(url_template):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/142.0.0.0 Safari/537.36"
    }

    page = 1
    while True:
        url = url_template.format(page)
        resp = requests.get(url, headers=headers)
        if resp.status_code != 200:
            break

        soup = BeautifulSoup(resp.text, "html.parser")
        listings = soup.find_all("a", class_="realTitle")

        if not listings:
            break  # hết page có listing

        page += 1

    return page - 1  # trả về tổng số page có dữ liệu

In [36]:
url = "https://nhadat.cafeland.vn/nha-dat-ban-tai-son-la/?page={}"

total_pages = get_total_pages_dynamic(url)

crawl_cafeland_region(url, total_pages)

Fetching page 1 ...
Processing https://nhadat.cafeland.vn/chinh-chu-gui-ban-biet-thu-4-tang-tai-trung-tam-thi-xa-moc-chau-2261963.html
Processing https://nhadat.cafeland.vn/chinh-chu-can-ban-lo-dat-5400m2-gan-thac-dai-yem-moc-chau-2261962.html
Processing https://nhadat.cafeland.vn/rui-ro-phap-ly-khong-so-do-san-sang-trao-tay-2260645.html
Processing https://nhadat.cafeland.vn/6-ty-can-xe-khe-105m2-3-tang-1-tum-tai-trung-tam-tt-moc-chau-son-la-2259064.html
Processing https://nhadat.cafeland.vn/chinh-chu-can-ban-suat-ngoai-giao-panacea-hill-105m2-huong-tay-bac-full-so-2258409.html
Processing https://nhadat.cafeland.vn/chinh-chu-ban-hon-6-ha-dat-co-tho-cu-tai-van-ho-2247757.html
Processing https://nhadat.cafeland.vn/chinh-chu-ban-3400-m2-co-200-tho-cu-tk-chieng-di-ttnt-moc-chau-son-la-2205207.html
Processing https://nhadat.cafeland.vn/chinh-chu-can-ban-755-m2-trung-tam-thi-tran-nong-truong-moc-chau-son-la-2205208.html
Processing https://nhadat.cafeland.vn/ban-mat-duong-ql6-450-m2-co-200-th